In [ ]:
# from vllm import LLM, SamplingParams
# from vllm.assets.image import ImageAsset
# from vllm.assets.video import VideoAsset
# from vllm.utils import FlexibleArgumentParser
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

from openai import OpenAI

# Modify OpenAI's API key and API base to use vLLM's API server.
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)


models = client.models.list()
model_path = models.data[0].id
print(f'model_path: {model_path}')
# model_path = "/DATA/disk1/ceyao/models/Qwen/Qwen2.5-VL-7B-Instruct"

In [ ]:
import base64
import requests

def encode_base64_content_from_url(content_url: str) -> str:
    """Encode a content retrieved from a remote url to base64 format."""

    with requests.get(content_url) as response:
        response.raise_for_status()
        result = base64.b64encode(response.content).decode('utf-8')

    return result

from utils import encode_image_to_base64, decode_base64_to_image
import matplotlib.pyplot as plt

from contextlib import contextmanager
from datetime import datetime
@contextmanager
def timer(name):
    start = datetime.now()
    yield
    end = datetime.now()
    print(f"{name} 执行时间: {(end - start).total_seconds():.2f} 秒")


from vis_bbox import plot_bounding_boxes, parse_json, reshape_img
from IPython.display import display
import ast
import json
from json_repair import repair_json
import json_repair

def inference(prompt, img_url=None, system_prompt="You are a helpful assistant", max_new_tokens=2048):
    
    image = Image.open(img_url)
    w, h = image.size

    img_base64 = encode_image_to_base64(image)

    
    chat_completion = client.chat.completions.create(
        model=model_path,
        max_completion_tokens=max_new_tokens,
        messages=[{
            "role": "user",
            "content": [
                {
                    "type": "text",
                    # "text": "What's in this image?"
                    "text": prompt
                },
                {
                    "type": "image_url",
                    "image_url": {
                        # "url": "https://q9.itc.cn/images01/20240529/091866645b3c485fb1a493ecbe302e06.png"
                        "url": "https://pics4.baidu.com/feed/b999a9014c086e06cff1723bb02d58fb0ad1cb1e.jpeg"
                        # "url": f"data:image/jpeg;base64,{image_base64}"
                        # "url": f"data:image/png;base64,{img_base64}"
                    },
                },
            ],
        }],
    )
    
    # b = decode_base64_to_image(a1)
    # plt.figure(1, figsize=(10,10))
    # plt.axis('off')
    # plt.imshow(b)
    # plt.show()
    prompt_tokens = chat_completion.usage.prompt_tokens
    completion_tokens = chat_completion.usage.completion_tokens
    total_tokens = chat_completion.usage.total_tokens

    print(f"Prompt tokens: {prompt_tokens}")
    print(f"Completion tokens: {completion_tokens}")
    print(f"Total tokens: {total_tokens}")

    response = chat_completion.choices[0].message.content
    print("Output:", response)

    return response, h, w



In [ ]:
# image_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"

# image_path = '/home/ceyao/Projects/DexGraspVLA-planner/planner-test/Gfp-wisconsin-madison-the-nature-boardwalk.jpg'
# image_path = '/home/ceyao/Projects/DexGraspVLA-planner/planner-test/002_The_lion_king_Snyggve_in_the_Serengeti_National_Park_Photo_by_Giles_Laurent.jpg'
image_path = '/home/ceyao/Projects/DexGraspVLA-planner/planner-test/2015_Kaczka_krzyżowka_w_wodzie_(samiec).jpg'
# image_path = '/home/ceyao/Projects/DexGraspVLA-planner/planner-test/white/2_Color.png'
# image_path = '/home/ceyao/Projects/DexGraspVLA-planner/planner-test/mid.png'
# image_path = '/home/ceyao/Projects/DexGraspVLA-planner/planner-test/high.png'



# image_path = reshape_img(image_path)

# image_base64 = encode_base64_content_from_url(image_url)
# image_path = '/home/ceyao/Projects/DexGraspVLA-planner/planner-test/low_quality_image.jpg'
from PIL import Image

img = Image.open(image_path)
img
# img.save('low_quality_image.jpg', quality=30)



In [ ]:


# prompt = "框出桌面上所有的绿色物品的位置，并输出其坐标。"
# prompt = "请以JSON格式输出图中所有绿色物品bbox的的坐标以及它们的名字和描述（例如红色盖子的瓶子，蓝色的玩偶），然后基于检测结果回答以下问题：图中物体的数目是多少？"
# prompt = "请以JSON格式输出图中所有包含绿色区域物品的bbox的坐标以及它们的名字和描述（例如红色盖子的瓶子，蓝色的玩偶）"
# prompt = "请以JSON格式输出图中所有物品的bbox的坐标以及它们的名字和描述（例如红色盖子的瓶子，蓝色的玩偶）"
# prompt = "请以JSON格式输出图中（除了左边白色塑料盒以外的）所有物品的bbox的坐标以及它们的名字和描述（例如红色盖子的瓶子，蓝色的玩偶）"
# prompt = "框出桌面上（不包括桌面下方以及凹陷处）所有的绿色物品的位置，并输出其坐标。"
prompt = "描述图片"

with timer("API 请求"):
    response, input_height, input_width = inference(prompt, image_path)


# # Parsing out the markdown fencing
# bounding_boxes = parse_json(response)
# try:
#     json_output = ast.literal_eval(bounding_boxes)
# except Exception as e:
#     # print(f"JSON 格式错误: {e}")
#     # end_idx = bounding_boxes.rfind('"}') + len('"}')
#     # truncated_text = bounding_boxes[:end_idx] + "]"
#     # json_output = ast.literal_eval(truncated_text)
#     # fixed_boxes = bounding_boxes.replace('"label: "', '"label": "')
#     # bounding_boxes = repair_json(bounding_boxes, ensure_ascii=False)
#     # json_output = json.loads(bounding_boxes)
#     json_output = json_repair.loads(bounding_boxes)

# print("\n\njson output:\n", json_output)

# # plot_bounding_boxes(image, response, input_width, input_height)
# # # 获取处理后的图像并显示
# result_img = plot_bounding_boxes(Image.open(image_path), json_output, input_width, input_height)
# new_img_path = image_path[:-4]+"_25-72B_bbox.png"
# result_img.save(new_img_path)

# display(result_img)  # Jupyter 会自动显示图像

In [3]:
a = os.getenv("DASHSCOPE_API_KEY")
print(a)

None


In [ ]:
import os
from openai import OpenAI

# Modify OpenAI's API key and API base to use vLLM's API server.
# openai_api_key = "EMPTY"
# openai_api_base = "http://localhost:8001/v1"
# openai_api_key = os.getenv("DASHSCOPE_API_KEY") # not work in Jupyter
openai_api_key = "sk-96e8a9444a5e48ea9cbc83d94f647239"
openai_api_base = "https://dashscope.aliyuncs.com/compatible-mode/v1"


client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)


models = client.models.list()
model_path = models.data[0].id
print(f'model_path: {model_path}')

# 查看API有哪些模型
# for model in models.data:
for model in models:
    print(model.id)

model_path: deepseek-v3
deepseek-v3
deepseek-r1-distill-llama-70b
deepseek-r1-distill-qwen-32b
deepseek-r1-distill-qwen-14b
deepseek-r1-distill-llama-8b
deepseek-r1-distill-qwen-1.5b
deepseek-r1-distill-qwen-7b
deepseek-r1
qwen1.5-7b-chat
qwen-vl-ocr-latest
qwen-vl-ocr
qwen-coder-plus-1106
qwen-coder-plus
qwen-coder-plus-latest
qwen2.5-coder-3b-instruct
qwen2.5-coder-0.5b-instruct
qwen2.5-coder-14b-instruct
qwen2.5-coder-32b-instruct
qwen-coder-turbo-0919
qwen2.5-0.5b-instruct
qwen2.5-1.5b-instruct
qwen2.5-3b-instruct
qwen2.5-7b-instruct
qwen2.5-14b-instruct
qwen2.5-32b-instruct
qwen2.5-72b-instruct
qwen2.5-coder-7b-instruct
qwen2.5-math-1.5b-instruct
qwen2.5-math-7b-instruct
qwen2.5-math-72b-instruct
qwen-turbo-0919
qwen-turbo-latest
qwen-plus-0919
qwen-plus-latest
qwen-max-0919
qwen-max-latest
qwen-coder-turbo
qwen-coder-turbo-latest
qwen-math-turbo-0919
qwen-math-turbo
qwen-math-turbo-latest
qwen-math-plus-0919
qwen-math-plus
qwen-math-plus-latest
qwen2-1.5b-instruct
qwen2-57b-a14b-

[Model(id='deepseek-v3', created=1740023101, object='model', owned_by='system'),
 Model(id='deepseek-r1-distill-llama-70b', created=1739966973, object='model', owned_by='system'),
 Model(id='deepseek-r1-distill-qwen-32b', created=1739966940, object='model', owned_by='system'),
 Model(id='deepseek-r1-distill-qwen-14b', created=1739966931, object='model', owned_by='system'),
 Model(id='deepseek-r1-distill-llama-8b', created=1739966898, object='model', owned_by='system'),
 Model(id='deepseek-r1-distill-qwen-1.5b', created=1739966882, object='model', owned_by='system'),
 Model(id='deepseek-r1-distill-qwen-7b', created=1739966868, object='model', owned_by='system'),
 Model(id='deepseek-r1', created=1739966853, object='model', owned_by='system'),
 Model(id='qwen1.5-7b-chat', created=1735036190, object='model', owned_by='system'),
 Model(id='qwen-vl-ocr-latest', created=1732693133, object='model', owned_by='system'),
 Model(id='qwen-vl-ocr', created=1732693123, object='model', owned_by='syste

In [28]:
# model_path = "qwen-vl-max"
# model_path = "qwen-vl-max-0125" # qwen2.5-vl
# model_path = "qwen-vl-max-1230" 
# model_path = "qwen-vl-chat-v1" # 说是一个文本模型
model_path = "qwen-vl-v1"

# image_url_duck = "https://upload.wikimedia.org/wikipedia/commons/d/da/2015_Kaczka_krzy%C5%BCowka_w_wodzie_%28samiec%29.jpg"
# image_url_lion = "https://upload.wikimedia.org/wikipedia/commons/7/77/002_The_lion_king_Snyggve_in_the_Serengeti_National_Park_Photo_by_Giles_Laurent.jpg"

image_url_duck = "https://q9.itc.cn/images01/20240529/091866645b3c485fb1a493ecbe302e06.png"
image_url_lion = "https://pics4.baidu.com/feed/b999a9014c086e06cff1723bb02d58fb0ad1cb1e.jpeg"

messages=[
        {
            "role": "system",
            "content": [{"type": "text", "text": "You are a helpful assistant."}],
        },
        {
        "role": "user",
        "content": [
            {
                "type": "text", 
                # "text": "What are the animals in these images?"
                # "text": "给图片中的动物打锚框，输出起bbox，用（x,y,x,y）的格式"
                "text": "识别图片中是动物，将其框出以json格式返回起bbox坐标，并回答是什么动物。"
            },
            # {
            #     "type": "image_url",
            #     "image_url": {
            #         "url": image_url_duck
            #         # "url": f"data:image/png;base64,{a}"
            #     },
            # },
            # {
            #     "type": "image_url",
            #     "image_url": {
            #         "url": image_url_lion
            #         # "url": f"data:image/png;base64,{a1}"
            #     },
            # },
        ],
    }
]

chat_completion = client.chat.completions.create(
    model=model_path,
    max_completion_tokens=64,
    messages=messages
    
)

prompt_tokens = chat_completion.usage.prompt_tokens
completion_tokens = chat_completion.usage.completion_tokens
total_tokens = chat_completion.usage.total_tokens

print(f"Prompt tokens: {prompt_tokens}")
print(f"Completion tokens: {completion_tokens}")
print(f"Total tokens: {total_tokens}")


assistant_message = chat_completion.choices[0].message
messages.append(assistant_message.model_dump())

response = chat_completion.choices[0].message.content
print("Output:", response)


Prompt tokens: 40
Completion tokens: 725
Total tokens: 765
Output: 作为一个文本模型，我无法直接识别和框出图片中的动物。你需要使用图像识别技术，如深度学习模型，来进行这个任务。这些模型通常需要大量的标注数据来训练，以便准确地识别图片中的对象。

然而，如果你已经有了这样的数据集，你可以使用这些数据集来训练模型，并使用这些模型来识别新的图片。一旦模型被训练，你就可以将图片输入到模型中，模型会输出一个或多个物体的边界框和预测的类别。

以下是一个简单的例子，展示了如何使用Python的PIL库和OpenCV库来识别图像中的动物：

```python
import cv2
import numpy as np
import json

def recognize_animal(image_path):
    # 读取图片
    image = cv2.imread(image_path)
    
    # 将图片转换为灰度
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # 应用高斯滤波以平滑图像
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # 将灰度图像转换为直方图
    histogram = np.sum(gray, axis=0)
    
    # 找到最大值的索引
    max_val = np.max(histogram)
    maxInd = np.unravel_index(np.argmax(histogram, axis=None), histogram.shape)
    
    # 计算最大值对应的通道位置
    maxInd = maxInd[1] * 3 + maxInd[0]
    
    # 将最大值对应的通道从直方图中提取出来
    maxPixel = np.copy(histogram)
    maxPixel[maxInd[0], maxInd[1]] = 0
    
    # 将直方图与最大值对应通道进行求和
    sumPixel = np.sum(maxPixel, axis=0)
  

In [9]:
response

'```json\n[\n\t{"bbox_2d": [14, 49, 620, 329], "label": "鸭子"}\n]\n```'

In [13]:
a = assistant_message.model_dump()
a

{'content': 'The animal in the image is a male mallard duck. Mallards are easily recognizable by their distinctive plumage, with males having a glossy green head, a white ring around the neck, a yellow bill, and a mix of gray and brown on the body.',
 'refusal': None,
 'role': 'assistant',
 'audio': None,
 'function_call': None,
 'tool_calls': None}

In [20]:
type(b)

str

In [19]:
b = chat_completion.model_dump_json()
b

'{"id":"chatcmpl-d60661c1-2216-9184-9383-214a68b146a8","choices":[{"finish_reason":"stop","index":0,"logprobs":null,"message":{"content":"The animal in the image is a male mallard duck. Mallards are easily recognizable by their distinctive plumage, with males having a glossy green head, a white ring around the neck, a yellow bill, and a mix of gray and brown on the body.","refusal":null,"role":"assistant","audio":null,"function_call":null,"tool_calls":null}}],"created":1739974851,"model":"qwen-vl-max","object":"chat.completion","service_tier":null,"system_fingerprint":null,"usage":{"completion_tokens":53,"prompt_tokens":429,"total_tokens":482,"completion_tokens_details":null,"prompt_tokens_details":null}}'

In [ ]:
completion = client.completions.create(model=model_path,
                                      prompt="San Francisco is a")
print("Completion result:\n", completion.choices[0].text)

In [ ]:
chat_response = client.chat.completions.create(
    model=model_path,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Tell me a joke."},
    ]
)
print("Chat response:\n", chat_response.choices[0].message.content)

In [5]:
from openai import OpenAI
from pathlib import Path
import base64, mimetypes, time

client = OpenAI(api_key="EMPTY", base_url="http://127.0.0.1:8001/v1")
model_id = client.models.list().data[0].id
print("Using model:", model_id)

# 从 z_multi_0905 选取连续5张图片，按文件名排序取前5张
images_dir = Path("data/multi_test_origin").resolve()
seq = [images_dir / f"{i}_0.png" for i in range(1546,1551)]
print("Files:", [p.name for p in seq])

# 转为 data URL 列表
image_urls = []
for p in seq:
    mime, _ = mimetypes.guess_type(str(p))
    if mime is None:
        mime = "application/octet-stream"
    with open(p, "rb") as f:
        b64 = base64.b64encode(f.read()).decode("utf-8")
    image_urls.append(f"data:{mime};base64,{b64}")

# 回合（多图）并测延时
start = time.perf_counter()
resp_huihe = client.chat.completions.create(
    model=model_id,
    messages=[
        {
            "role": "system",
            "content": [{
                "type": "text",
                "text": (
                    "You are a professional Mahjong AI assistant. You need to carefully analyze the Mahjong scene in the image—including the wall, the discard area, and the players' hand movements—and then precisely determine whose turn it is now. "
                    "Your answer must be and can only be one of the following: myself, next, opposite, previous, or unknown. Do not output anything else."
                )
            }],
        },
        {
            "role": "user",
            "content": (
                [{"type": "image_url", "image_url": {"url": url}} for url in image_urls]
                + [{"type": "text", "text": "Whose turn is it now?"}]
            ),
        },
    ],
    temperature=0.95,
    max_tokens=128,
)
elapsed_ms = (time.perf_counter() - start) * 1000.0
print(f"Latency (huihe, 5 imgs): {elapsed_ms:.2f} ms")
print("Assistant:", resp_huihe.choices[0].message.content)

Using model: /share_data/yanruilin/saves/qwen2_5vl-7b/full/sft/majiang_turn_1016_merge6/checkpoint-500
Files: ['1546_0.png', '1547_0.png', '1548_0.png', '1549_0.png', '1550_0.png']
Latency (huihe, 5 imgs): 298.39 ms
Assistant: myself
